NLP Assignment 2

Group 15:
Ahmed 23PGAI0120
Akash Deshwani 23PGAI0035
Harshada Suresh Jadhav 23PGGAI0101
Rohan Mehta 23PGAI0001

In [7]:
import tkinter as tk
import requests
import re
import numpy as np
import nltk
import tkinter as tk
from bs4 import BeautifulSoup
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

def get_recipe_name(url):
    if url.split('/')[-1] != '':
        return(url.split('/')[-1])
    else:
        return(url.split('/')[-2])

def scrape_web_page(query):
    link = 'https://www.allrecipes.com/search?q='
    query = query.replace(" ", "-")
    find_recipe = requests.get(link+query)
    soup = BeautifulSoup(find_recipe.content, 'html.parser')
    s = soup.find('div', class_='comp search-results__content mntl-block')
    content = s.find_all('a')
    recipe_link = content[0].get('href')
    source = requests.get(recipe_link).text
    return(BeautifulSoup(source, 'html.parser'))

def get_ingredients(source_text, tag, class_name):
    temp_ingredients_1 = source_text.find(tag,class_=class_name).text
    temp_ingredients_2 = temp_ingredients_1.split('\n')
    ingredients = []
    for i in range(len(temp_ingredients_2)):
        if temp_ingredients_2[i]:
            a = temp_ingredients_2[i].strip()
            a = a.replace('⅔', '0.67')
            a = a.replace('½', '0.5')
            a = a.replace('⅓', '0.34')
            a = a.replace('¼', '0.25')
            a = a.replace('⅛', '0.13')
            ingredients.append(a.lower())
    return(ingredients[1:])

def get_steps(source_text, tag, class_name):
    temp_steps_1 = source_text.find(tag,class_=class_name).text
    temp_steps_2 = temp_steps_1.split('\n')
    temp_steps_3 = []
    for i in range(len(temp_steps_2)):
        if temp_steps_2[i]:
            a = temp_steps_2[i].strip()
            temp_steps_3.append(a)
    temp_steps_3 = temp_steps_3[1:]
    steps = []
    for i in range(len(temp_steps_3)):
        if '.' in temp_steps_3[i]:
            a = []
            a = temp_steps_3[i].split('.')
            for j in range(len(a)):
                if a[j]:
                    steps.append(a[j].strip().lower())
        else:
            steps.append(temp_steps_3[i].strip().lower())
    return(steps[1:])

def get_tools(abc):
    tool_list = ['baking sheet', 'bakingsheet', 'knife', 'mould', 'blender', 'bottle opener', 'bottleopener', 'bowl', 'tray', 'tawa', 'cooker', 'pressure cooker', 'belan', 'rolling pin', 'microwave', 'spatula', 'tongs', 'grinder', 'can opener', 'canopener', 'thermometer', 'grater', 'pot', 'cup', 'cutting board', 'cuttingboard', 'spoon', 'tablespoon', 'dish', 'sieve', 'whisk', 'frying pan', 'pan', 'funnel', 'strainer', 'chopper', 'scoop', 'mallet', 'scissors', 'ladle', 'squeezer', 'fork', 'mincer', 'nutcracker', 'oven glove', 'oven mitt', 'oven', 'pan', 'blender', 'peeler', 'cutter', 'masher', 'saucepan', 'skillet', 'stove', 'teaspoon', 'tin opener', 'can opener']
    tools = []
    for k in abc:
        j = k.lower()   
        for a in tool_list:
            if a in j:
                tools.append(a)
    return(list(np.unique(tools)))

def get_temp(abc, bcd):
    temp_list = ['degrees', 'Degrees', 'temperature', 'Temperature', 'Celcius', 'celcius', 'Kelvin', 'kelvin', 'C ', 'F ']
    temp = []
    for k in bcd:
        j = k.lower()   
        for a in temp_list:
            if a in j:
                temp.append(k)
    return(temp)

def get_quantity(abc, bcd):
    tokenized = sent_tokenize(abc)
    for i in tokenized:
        wordsList = nltk.word_tokenize(i)
        tagged = nltk.pos_tag(wordsList)
    abcd = []
    abcde=[]
    abcdef = []
    question = ['much', 'Much', 'quantity', 'Quantity', 'measurement', 'Measurement', 'portion', 'Portion']
    qflag = 1
    tfound =1
    sfound = 1
    if (len(abc.split(' ')) > 3):
        for a in question:
            if a in abc:
                qflag = 0
                for i in range(len(tagged)-2):
                    abcde.append((tagged[i][0],tagged[i][1],tagged[i+1][0],tagged[i+1][1],tagged[i+2][0],tagged[i+2][1]))
        for i in range(len(abcde)):
            if (abcde[i][1] in ('JJ','NN', 'NNS', 'NNP', 'NNPS')) and (abcde[i][3] in ('JJ','NN', 'NNS', 'NNP', 'NNPS')) and (abcde[i][5] in ('NN', 'NNS', 'NNP', 'NNPS')):
                abcdef.append(abcde[i][0]+ ' ' + abcde[i][2]+ ' ' + abcde[i][4])
                tfound = 0
    if (tfound == 1):
        for a in question:
            if a in abc:
                qflag = 0
                for i in range(len(tagged)-1):
                    abcde.append((tagged[i][0],tagged[i][1],tagged[i+1][0],tagged[i+1][1]))
        for i in range(len(abcde)):
            if (abcde[i][1] in ('JJ','NN', 'NNS', 'NNP', 'NNPS')) and (abcde[i][3] in ('NN', 'NNS', 'NNP', 'NNPS')):
                abcdef.append(abcde[i][0]+ ' ' + abcde[i][2])
                sfound = 0
    if ((tfound == 1) and (sfound ==1)):
        for a in question:
            if a in abc:
                qflag = 0
                for i in range(len(tagged)):
                    abcde.append((tagged[i][0],tagged[i][1]))
        for i in range(len(abcde)):
            if (abcde[i][1] in ('NN', 'NNS', 'NNP', 'NNPS')):
                abcdef.append(abcde[i][0])
    if (qflag == 1):
        return('Please rephrase your question')
    fflag = 1
    ing = []
    for i in range(len(abcdef)):
        for j in range(len(bcd)):
            if abcdef[i] in bcd[j]:
                ing.append(bcd[j])
                fflag =0
    if fflag ==0:
        return (ing)
#     if fflag == 1:
#         return('Ingredient not found')

def get_previous(abc, bcd):
    stop_words = stopwords.words('english')
    abc = abc.lower()
    words = word_tokenize(abc)
    remove_list = ['previous', 'step', 'process', 'activity', 'before', '?']
    words = [word for word in words if word not in stop_words]
    words = [word for word in words if word not in remove_list]
    a = str('')
    for i in range(len(words)):
        a = a + ' ' + words[i]
    a=a.strip()
    found = 0
    for i in range(len(bcd)):
        if a in bcd[i]:
            found = 1
            break
    if i == 0:
        return('No Previous Step')
    else:
        if found == 1:
            return('Previous Step: ', bcd[i-1])
        else:
            return('Could not find step')

def get_next(abc, bcd):
    stop_words = stopwords.words('english')
    abc = abc.lower()
    words = word_tokenize(abc)
    remove_list = ['next', 'step', 'process', 'activity', 'after', '?']
    words = [word for word in words if word not in stop_words]
    words = [word for word in words if word not in remove_list]
    a = str('')
    for i in range(len(words)):
        a = a + ' ' + words[i]
    a=a.strip()
    found = 0
    for i in range(len(bcd)):
        if a in bcd[i]:
            found = 1
            break
    if i == len(bcd):
        return('No Next Step')
    else:
        if found == 1:
            return('Next Step: ', bcd[i+1])
        else:
            return('Could not find step')

def get_prep_time(source_text):
    a=source_text.find_all('div',class_="mntl-recipe-details__label")[0].text
    b=source_text.find_all('div',class_="mntl-recipe-details__value")[0].text
    c = "{} {}".format(a,b)
    return(c)

def get_cook_time(source_text):
    a=source_text.find_all('div',class_="mntl-recipe-details__label")[1].text
    b=source_text.find_all('div',class_="mntl-recipe-details__value")[1].text
    c = "{} {}".format(a,b)
    return(c)

def get_total_time(source_text):
    a=source_text.find_all('div',class_="mntl-recipe-details__label")[2].text
    b=source_text.find_all('div',class_="mntl-recipe-details__value")[2].text
    c = "{} {}".format(a,b)
    return(c)

def get_servings(source_text):
    a=source_text.find_all('div',class_="mntl-recipe-details__label")[3].text
    b=source_text.find_all('div',class_="mntl-recipe-details__value")[3].text
    c = "{} {}".format(a,b)
    return(c)

def question_selector(abc, bcd, defg, source_text):
    quantity = ['much', 'Much', 'quantity', 'Quantity', 'measurement', 'Measurement', 'portion', 'Portion']
    tools = ['tool', 'tools', 'Tool', 'Tools']
    temp_list = ['degrees', 'Degrees', 'temperature', 'Temperature', 'Celcius', 'celcius', 'Kelvin', 'kelvin', 'C ', 'F ']
    prev_list = ['previous', 'before']
    next_list = ['next', 'after']
    ingredient_list = ['ingredient', 'ingredients', 'Ingredient', 'Ingredients']
    steps_list = ['processes', 'Processes', 'activities', 'Activities', 'steps', 'Steps']
    serves_list = ['serves', 'serve']
    for a in tools:
            if a in abc:
                a = get_tools(bcd)
                for i in range(len(a)):
                    print(a[i])
                break
    for a in quantity:
        if a in abc:
            b = get_quantity(abc, defg)
            print(b)
            break
    for a in temp_list:
        if a in abc:
            b = get_temp(abc, bcd)
            for i in range(len(b)):
                print(b[i])
            break
    for a in prev_list:
        if a in abc:
            b = get_previous(abc, bcd)
            print(b)
            break    
    for a in next_list:
        if a in abc:
            b = get_next(abc, bcd)
            print(b)
            break
    for a in ingredient_list:        
        if a in abc:
            for i in range(len(defg)):
                print(defg[i])
            break
    for a in steps_list:        
        if a in abc:
            for i in range(len(bcd)):
                print(bcd[i])
            break
    if 'preparation' in abc:
        print(get_prep_time(source_text))
    if 'cooking' in abc:
        print(get_cook_time(source_text))
    if 'total' in abc:
        print(get_total_time(source_text))
    for a in serves_list:        
        if a in abc:
            print(get_servings(source_text))

def trial_question_selector(abc, bcd, defg, source_text):
    quantity = ['much', 'Much', 'quantity', 'Quantity', 'measurement', 'Measurement', 'portion', 'Portion']
    tools = ['tool', 'tools', 'Tool', 'Tools']
    temp_list = ['degrees', 'Degrees', 'temperature', 'Temperature', 'Celcius', 'celcius', 'Kelvin', 'kelvin', 'C ', 'F ']
    prev_list = ['previous', 'before']
    next_list = ['next', 'after']
    ingredient_list = ['ingredient', 'ingredients', 'Ingredient', 'Ingredients']
    steps_list = ['processes', 'Processes', 'activities', 'Activities', 'steps', 'Steps']
    serves_list = ['serves', 'serve']
    for a in tools:
            if a in abc:
                return(get_tools(bcd))
                break
    for a in quantity:
        if a in abc:
            return(get_quantity(abc, defg))
            break
    for a in temp_list:
        if a in abc:
            return(get_temp(abc, bcd))
            break
    for a in prev_list:
        if a in abc:
            return(get_previous(abc, bcd))
            break    
    for a in next_list:
        if a in abc:
            return(get_next(abc, bcd))
            break
    for a in ingredient_list:        
        if a in abc:
            return(defg)
    for a in steps_list:        
        if a in abc:
            return(bcd)
    if 'preparation' in abc:
        return(get_prep_time(source_text))
    if 'cooking' in abc:
        return(get_cook_time(source_text))
    if 'total' in abc:
        return(get_total_time(source_text))
    for a in serves_list:        
        if a in abc:
            return(get_servings(source_text))

def assignment(url):

    recipe_name = get_recipe_name(url)
    source_text = scrape_web_page(url)
    ingredients = get_ingredients(source_text, 'div', 'comp mntl-structured-ingredients')
    steps = get_steps(source_text, 'div', 'comp recipe__steps mntl-block')
    tools = get_tools(steps)

    recipe = {}
    recipe['name'] = recipe_name
    recipe['ingredients'] = ingredients
    recipe['directions'] = steps
    recipe['tools'] = tools

    return(recipe, source_text)

window = tk.Tk()
window.title("Interactive Cookbook")
window.geometry("800x800")
label1 = tk.Label(text="Input Recipe")
label1.pack()
sample_text1 = tk.Text(height = 1.5)
sample_text1.pack()

def search_button():
    global a
    global b
    global source_text
    a = sample_text1.get("1.0", tk.END)
    b, source_text = assignment(a)

search_button11 = tk.Button(window, height=1, width=10, text="Search", command=search_button)
search_button11.pack()
label2 = tk.Label(text="Input Query")
label2.pack()
sample_text2 = tk.Text(height = 1.5)
sample_text2.pack()

def set_text_by_button():
    a1 = sample_text2.get("1.0", tk.END)
    b1 = trial_question_selector(a1, b['directions'], b['ingredients'], source_text)
    sample_text3.delete("1.0","end")
    sample_text3.insert('1.0', b1)
set_up_button = tk.Button(window, height=1, width=10, text="Set", command=set_text_by_button)
set_up_button.pack()
label3 = tk.Label(text="Answer")
label3.pack()
sample_text3 = tk.Text()
sample_text3.pack()
que = ['You can ask these kind of questions.\n', 'What are the ingredients required for this recipe?', '\nWhat are the steps to be followed for this recipe?', '\nWhat are the tools required for this recipe?', '\nWhat is the preparation time for this recipe?', '\nWhat is the cooking time for this recipe?', '\nWhat is the total time for this recipe?', '\nHow many people does this recipe serve?', '\nMention all temperatures', '\nWhat is the previous step of place pork sections?', '\nWhat is the next step of place pork sections?', '\nWhat is the quantity of chinese rice wine?', '\nWhat is the quantity of rice wine?', '\nWhat is the quantity of wine?']
def delete():
    sample_text3.delete("1.0","end")
    sample_text3.insert('1.0', que)
b1= tk.Button(window, text= "Refresh",command= delete)
b1.pack()
window.mainloop()


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\coola\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\coola\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\coola\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Exception in Tkinter callback
Traceback (most recent call last):
  File "C:\Users\coola\anaconda3\lib\tkinter\__init__.py", line 1892, in __call__
    return self.func(*args)
  File "C:\Users\coola\AppData\Local\Temp\ipykernel_5624\1556197652.py", line 358, in set_text_by_button
    sample_text3.insert('1.0', b1)
  File "C:\Users\coola\anaconda3\lib\tkinter\__init__.py", line 3743, in insert
    self.tk.call((self._w, 'insert', index, chars) + args)
_tkinter.TclError: wr